In [1]:
%%writefile going_modular/data_setup.py

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

NUM_WORKER = os.cpu_count()

def create_dataloader(
    train_dir: str,
    test_dir: str,
    transforms: transforms.Compose,
    batch_size: int,
):
    train_data = datasets.ImageFolder(
        root = train_dir,
        transform = transforms,
        ) 
    test_data = datasets.ImageFolder(
        root = test_dir,
        transform = transforms
    )
    
    
    class_names = train_data.classes
    
    train_dataloader = DataLoader(
        dataset= train_data,
        batch_size= batch_size,
        shuffle= True,
        num_workers= NUM_WORKER,
        pin_memory= True
    )
    
    test_dataloader = DataLoader(
        dataset=test_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=NUM_WORKER,
        pin_memory=True
    )
    
    return train_dataloader, test_dataloader, class_names

Overwriting going_modular/data_setup.py


In [1]:
import torch
import torch.nn as nn

class TinyVGG(nn.Module):
    def __init__(self, in_channels:int, out_shape:int, hidden_units:int):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.BatchNorm2d(num_features=hidden_units),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride = 1,
                      padding=1),
            nn.BatchNorm2d(num_features=hidden_units),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride = 2))
        self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*13*13,
                      out_features=out_shape)
        )
        
    def forward(self, X):
        X = self.conv_block_1(X),
        # X = self.conv_block_2(X)
        # X = self.classifier(X)
        return X

In [2]:
a = torch.randn((32,3, 64, 64))

In [3]:
y = TinyVGG(3,4,8)

In [4]:
y_pred = y(a)

In [9]:
a=y_pred